## import necessary packages 

In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

## read in CSVs and check first few rows 

In [2]:
calls_derecho = pd.read_csv('./data/911_Phone_Calls_Derecho_050320_050420.csv')
calls_tornado = pd.read_csv('./data/911_Phone_Calls_Tornado_030320.csv')
dispatch_derecho = pd.read_csv('./data/Computer_Aided_Dispatch_Data_Derecho_Incidents_050320-050420.csv', skiprows = 3)
dispatch_tornado = pd.read_csv('./data/Computer_Aided_Dispatch_Data_Tornado_Incidents_030320.csv', skiprows = 3)

## calls_derecho

In [3]:
calls_derecho.head()

,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address
0,5/3/2020 16:30,911 Calls,36.247802,-86.716847,WIRELESS CALLER
1,5/3/2020 16:30,911 Calls,36.143108,-86.800621,1161 21ST AV S - SE
2,5/3/2020 16:30,911 Calls,36.100516,-87.056329,8013 C MCCRORY LN - N
3,5/3/2020 16:30,911 Calls,36.130021,-86.927819,5758 RIVER RD - SW
4,5/3/2020 16:31,911 Calls,36.142380,-86.881882,3744B ANNEX AVE - SE


In [4]:
calls_derecho = calls_derecho.rename(columns = {'Seizure DateTime': 'DateTime', 'CallTypeId': 'Call Type', 'ALI Latitude': 'Latitude', 'ALI Longitude': 'Longitude', 'Cell Tower Address': 'Cell Tower'})

In [5]:
calls_derecho['Geometry'] = calls_derecho.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

In [6]:
calls_derecho.head()

,DateTime,Call Type,Latitude,Longitude,Cell Tower,Geometry
0,5/3/2020 16:30,911 Calls,36.247802,-86.716847,WIRELESS CALLER,POINT (-86.716847 36.247802)
1,5/3/2020 16:30,911 Calls,36.143108,-86.800621,1161 21ST AV S - SE,POINT (-86.80062099999999 36.14310800000001)
2,5/3/2020 16:30,911 Calls,36.100516,-87.056329,8013 C MCCRORY LN - N,POINT (-87.05632900000001 36.100516)
3,5/3/2020 16:30,911 Calls,36.130021,-86.927819,5758 RIVER RD - SW,POINT (-86.927819 36.130021)
4,5/3/2020 16:31,911 Calls,36.142380,-86.881882,3744B ANNEX AVE - SE,POINT (-86.88188199999999 36.14238)


In [7]:
calls_derecho['Cell Tower'].value_counts()

WIRELESS CALLER                339
4400 HARDING PIKE - E           15
3333 LEBANON PIKE - N           14
3009D DARRINGTON WAY - N        14
3901 GRANNY WHITE PIKE - SE     14
                              ... 
859 GRANADA AVE - S              1
508 INTERSTATE BLVD S - SW       1
1608-B 8TH AV - SECTOR N N       1
707 3RD AVE S - SE SECTOR        1
1000C S CARTWRIGHT ST - SE       1
Name: Cell Tower, Length: 630, dtype: int64

In [8]:
calls_derecho['Cell Tower'].nunique()

630

In [9]:
calls_derecho.info

<bound method DataFrame.info of             DateTime  Call Type   Latitude  Longitude  \
0     5/3/2020 16:30  911 Calls  36.247802 -86.716847   
1     5/3/2020 16:30  911 Calls  36.143108 -86.800621   
2     5/3/2020 16:30  911 Calls  36.100516 -87.056329   
3     5/3/2020 16:30  911 Calls  36.130021 -86.927819   
4     5/3/2020 16:31  911 Calls  36.142380 -86.881882   
...              ...        ...        ...        ...   
1544   5/4/2020 0:18  911 Calls  36.131694 -86.770535   
1545   5/4/2020 0:22  911 Calls  36.042237 -86.743455   
1546   5/4/2020 0:25  911 Calls  36.077664 -86.655607   
1547   5/4/2020 0:29  911 Calls  36.024706 -86.743562   
1548   5/4/2020 0:29  911 Calls        NaN        NaN   

                               Cell Tower  \
0                         WIRELESS CALLER   
1                     1161 21ST AV S - SE   
2                   8013 C MCCRORY LN - N   
3                      5758 RIVER RD - SW   
4                    3744B ANNEX AVE - SE   
...          

In [10]:
calls_derecho.shape

(1549, 6)

In [11]:
calls_derecho.isnull().sum()

DateTime       0
Call Type      0
Latitude      66
Longitude     66
Cell Tower    24
Geometry       0
dtype: int64

## calls_tornado 

In [12]:
calls_tornado = calls_tornado.rename(columns = {'Seizure DateTime': 'DateTime', 'CallTypeId': 'Call Type', 'ALI Latitude': 'Latitude', 'ALI Longitude': 'Longitude', 'Cell Tower Address': 'Cell Tower'})

In [13]:
calls_tornado['Geometry'] = calls_tornado.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

In [14]:
calls_tornado.head()

,DateTime,Call Type,Latitude,Longitude,Cell Tower,Geometry
0,3/3/2020 0:34,911 Calls,36.072708,-86.665779,620 Richards RD - N Sector,POINT (-86.665779 36.072708)
1,3/3/2020 0:36,911 Calls,36.159634,-86.841904,4507D KENTUCKY AV - SE,POINT (-86.841904 36.159634)
2,3/3/2020 0:36,911 Calls,36.264153,-86.708862,329 THELMA ST - S Sector,POINT (-86.708862 36.264153)
3,3/3/2020 0:36,911 Calls,NaN,NaN,526 FOUNDRY DR,POINT (nan nan)
4,3/3/2020 0:36,911 Calls,36.233467,-86.721077,5007B RUSKIN AVE - N,POINT (-86.72107700000001 36.233467)


In [15]:
calls_tornado['Cell Tower'].value_counts()

WIRELESS CALLER                       101
907 12TH AV N - N                      16
1051G CHICAMAUGA AVE - S               10
3333 LEBANON PIKE - SE                  8
4940 OLD HICKORY BLVD. - SE SECTOR      7
                                     ... 
526 FOUNDRY DR                          1
5171 HICKORY HOLLOW PKY - SE            1
4190 NOLENSVILLE PKE                    1
1051G CHICAMAUGA AVE - NW               1
619 DUE WEST AVE - NE SECTOR            1
Name: Cell Tower, Length: 211, dtype: int64

In [16]:
calls_tornado['Cell Tower'].nunique()

211

In [17]:
calls_tornado.info

<bound method DataFrame.info of           DateTime  Call Type   Latitude  Longitude  \
0    3/3/2020 0:34  911 Calls  36.072708 -86.665779   
1    3/3/2020 0:36  911 Calls  36.159634 -86.841904   
2    3/3/2020 0:36  911 Calls  36.264153 -86.708862   
3    3/3/2020 0:36  911 Calls        NaN        NaN   
4    3/3/2020 0:36  911 Calls  36.233467 -86.721077   
..             ...        ...        ...        ...   
470  3/3/2020 4:54  911 Calls  36.142552 -86.635330   
471  3/3/2020 4:54  911 Calls  36.189415 -86.813557   
472  3/3/2020 4:55  911 Calls  36.223534 -86.757874   
473  3/3/2020 4:56  911 Calls  36.168398 -86.783774   
474  3/3/2020 4:58  911 Calls  36.276566 -86.799417   

                            Cell Tower                              Geometry  
0           620 Richards RD - N Sector          POINT (-86.665779 36.072708)  
1               4507D KENTUCKY AV - SE          POINT (-86.841904 36.159634)  
2             329 THELMA ST - S Sector          POINT (-86.708862 36.2

In [18]:
calls_tornado.shape

(475, 6)

In [19]:
calls_tornado.isnull().sum()

DateTime       0
Call Type      0
Latitude      25
Longitude     25
Cell Tower     7
Geometry       0
dtype: int64

## dispatch_derecho

In [20]:
dispatch_derecho = dispatch_derecho.rename(columns = {'IncidentDate': 'DateTime', 'Latitude1': 'Latitude', 'Longitude1': 'Longitude', 'IncidentTypeDescription1': 'Incident Description'})

In [21]:
dispatch_derecho['Geometry'] = dispatch_derecho.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

In [22]:
dispatch_derecho.head()

,DateTime,Location,Latitude,Longitude,Incident Description,Geometry
0,5/3/2020 4:31:07 PM,901 KENWICK CT W,36.046877,-86.962526,ELECTRICAL HAZARD,POINT (-86.96252641 36.04687715)
1,5/3/2020 4:32:18 PM,8036 ARBOR DR,36.082491,-86.940680,ELECTRICAL HAZARD,POINT (-86.94067952 36.08249081)
2,5/3/2020 4:32:23 PM,119 DUE WEST AV E,36.248029,-86.716875,NOT ALERT COPD,POINT (-86.71687484 36.24802948)
3,5/3/2020 4:33:04 PM,630 GALLATIN PKE S,36.256491,-86.714817,ABDOMINAL PAIN,POINT (-86.71481656 36.25649117)
4,5/3/2020 4:33:52 PM,615 W HILLWOOD DR,36.129690,-86.879834,ELECTRICAL HAZARD,POINT (-86.87983392 36.12968971)


In [23]:
dispatch_derecho['Incident Description'].value_counts()

ELECTRICAL HAZARD                              370
GENERAL ALARM COMMERCIAL   INDUSTRIAL BUILD     23
MVA INJURIES:                                    8
SMALL BUILDING INVESTIGATION                     6
UNKNOWN STATUS:                                  6
                                              ... 
RESIDENTIAL FIRE PERSON TRAPPED                  1
WATER PROBLEM WITH ELECTRICAL HAZARD             1
OVERDOSE - UNKNOWN STATUS                        1
PULL STATION ALARM HOUSE                         1
CHEST PAIN - ABNORMAL BREATHING                  1
Name: Incident Description, Length: 122, dtype: int64

In [24]:
dispatch_derecho['Incident Description'].nunique()

122

In [25]:
dispatch_derecho.info

<bound method DataFrame.info of                  DateTime            Location   Latitude  Longitude  \
0     5/3/2020 4:31:07 PM    901 KENWICK CT W  36.046877 -86.962526   
1     5/3/2020 4:32:18 PM       8036 ARBOR DR  36.082491 -86.940680   
2     5/3/2020 4:32:23 PM   119 DUE WEST AV E  36.248029 -86.716875   
3     5/3/2020 4:33:04 PM  630 GALLATIN PKE S  36.256491 -86.714817   
4     5/3/2020 4:33:52 PM   615 W HILLWOOD DR  36.129690 -86.879834   
..                    ...                 ...        ...        ...   
622  5/4/2020 12:21:00 AM       625 BENTON AV  36.131384 -86.770628   
623  5/4/2020 12:21:32 AM    518 BRENTLAWN DR  36.064362 -86.761199   
624  5/4/2020 12:22:47 AM       517 ALBANY DR  36.212754 -86.600431   
625  5/4/2020 12:27:52 AM    639 LAFAYETTE ST  36.153769 -86.777520   
626  5/4/2020 12:29:50 AM      2011 HAYES ST   36.151839 -86.800791   

                                 Incident Description  \
0                                   ELECTRICAL HAZARD   
1

In [26]:
dispatch_derecho.shape

(627, 6)

In [27]:
dispatch_derecho.isnull().sum()

DateTime                0
Location                0
Latitude                0
Longitude               0
Incident Description    0
Geometry                0
dtype: int64

## dispatch_tornado 

In [28]:
dispatch_tornado = dispatch_tornado.rename(columns = {'IncidentDate': 'DateTime', 'Latitude1': 'Latitude', 'Longitude1': 'Longitude', 'IncidentTypeDescription1': 'Incident Description'})

In [29]:
dispatch_tornado['Geometry'] = dispatch_tornado.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

In [30]:
dispatch_tornado.head()

,DateTime,Location,Latitude,Longitude,Incident Description,Geometry
0,3/3/2020 12:36:34 AM,6205 COCKRILL BEND CIR,36.182002,-86.894052,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.89405212 36.18200191)
1,3/3/2020 12:37:24 AM,7337 COCKRILL BEND BLVD,36.175666,-86.894978,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.89497756999999 36.175666)
2,3/3/2020 12:38:17 AM,110 TUNE AIRPORT DR,36.179299,-86.883727,STRUCTURE FIRE COMMERCIAL INDUSTRIAL,POINT (-86.8837274 36.17929946)
3,3/3/2020 12:39:40 AM,555 MARRIOTT DR,36.151108,-86.689921,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.6899214 36.1511077)
4,3/3/2020 12:39:42 AM,1414 COUNTY HOSPITAL RD,36.177926,-86.845934,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.84593411 36.17792604)


In [31]:
dispatch_tornado['Incident Description'].value_counts()

GENERAL ALARM COMMERCIAL   INDUSTRIAL BUILD    27
OUTSIDE ODOR                                   26
WIRES DOWN WITH NO SMOKE OR ARCING             17
SMALL BUILDING INVESTIGATION                   15
ELECTRICAL HAZARD WITH OR NEAR WATER           12
                                               ..
ELECTRICAL ARCING                               1
APARTMENT FIRE                                  1
ABNORMAL BREATHING                              1
POSSIBLY DANGEROUS HEMORRHAGE TRAUMA            1
UNKNOWN COMMERCIAL  INDUSTRIAL ALARM            1
Name: Incident Description, Length: 90, dtype: int64

In [32]:
dispatch_tornado['Incident Description'].nunique()

90

In [33]:
dispatch_tornado.info

<bound method DataFrame.info of                  DateTime                     Location   Latitude  Longitude  \
0    3/3/2020 12:36:34 AM      6205 COCKRILL BEND CIR   36.182002 -86.894052   
1    3/3/2020 12:37:24 AM     7337 COCKRILL BEND BLVD   36.175666 -86.894978   
2    3/3/2020 12:38:17 AM         110 TUNE AIRPORT DR   36.179299 -86.883727   
3    3/3/2020 12:39:40 AM             555 MARRIOTT DR   36.151108 -86.689921   
4    3/3/2020 12:39:42 AM     1414 COUNTY HOSPITAL RD   36.177926 -86.845934   
..                    ...                          ...        ...        ...   
255   3/3/2020 4:53:43 AM               1616 23RD AV N  36.175739 -86.813173   
256   3/3/2020 4:56:56 AM        5610 OLD HICKORY BLVD  36.180487 -86.601466   
257   3/3/2020 4:57:06 AM          2903 MORNINGSIDE DR  36.223554 -86.757949   
258   3/3/2020 4:57:14 AM               1730 KELLOW ST  36.189445 -86.813466   
259   3/3/2020 4:57:48 AM  I24 W / HICKORY HOLLOW PKWY  36.039720 -86.649099   

       

In [34]:
dispatch_tornado.shape

(260, 6)

In [35]:
dispatch_tornado.isnull().sum()

DateTime                0
Location                0
Latitude                0
Longitude               0
Incident Description    0
Geometry                0
dtype: int64